<a href="https://colab.research.google.com/github/RafaelKampa/ConstrucaoInterpretadores/blob/main/Matriz_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>TRABALHO RECUPERAÇÃO DE TEXTOS – TF-IDF</b></h1>
<h2>Alunos: Bernardo Zeni Diniz, Lucas Camargo Vianna, Pedro Leonardo Alves da Silva, Rafael Gilberto Kampa</h2>


<h1>Enunciado:</h1>
Sua tarefa será gerar a matriz termo documento, usando o algoritmo TF-IDF, dos documentos
recuperados da internet e imprimir esta matriz na tela. Para tanto:<br><br>
a) Seu algoritmo deve receber um link, para um documento txt, disponível na internet (Projeto Gutenberg - Free eBooks | Project Gutenberg), transformar todas as sentenças deste documento em listas de strings. As sentenças serão limitas por ponto final, dois pontos ou ponto e virgula.<br><br>
b) O conjunto de listas de sentenças será o corpus que você irá usar para criar a matriz termo documento usando o algoritmo bag of words.<br><br>
c) O resultado esperado será a apresentação da matriz termo documento em uma forma interativa que permita a visualização das linhas e colunas.

In [ ]:
#Gera um arquivo .txt com a matriz Bag of Words
import requests
import re
import numpy as np

# # Salva o nome do arquivo em uma variável global para futuros usos
nome_arquivo_gerado = "BagOfWords.txt"

# Lê o arquivo de texto do URL fornecido
response = requests.get("https://www.gutenberg.org/files/53106/53106-8.txt")
texto = response.text

# Separa o texto em documentos usando "." "," ";" como separadores e substitui "\n" e "\r" por um espaço em branco
separadores = [".", ",", ";"]
texto = texto.replace('\n', ' ').replace('\r', ' ')
documentos = re.split('|'.join(map(re.escape, separadores)), texto)

# # Salva o nome do arquivo em uma variável global para futuros usos
# nome_arquivo_gerado = "BagOfWords.txt"
# nome_arquivo_local = "teste.txt"

# # Lê o arquivo de texto local
# with open(nome_arquivo_local, "r") as file:
#     texto = file.read()

# # Separa o texto em documentos usando ".", ",", ";" como separadores e substitui "\n" e "\r" por um espaço em branco
# separadores = [".", ",", ";"]
# texto = texto.replace('\n', ' ').replace('\r', ' ')
# documentos = re.split('|'.join(map(re.escape, separadores)), texto)

# Remove os documentos vazios (resultantes de múltiplos caracteres separadores consecutivos)
documentos = list(filter(lambda x: len(x) > 0, documentos))

# Remove os espaços em branco do início e do final dos documentos
documentos = [documento.strip() for documento in documentos]

# Cria um set de todas as palavras encontradas nos documentos
todas_palavras = set()
for documento in documentos:
    palavras = documento.split()
    for palavra in palavras:
        todas_palavras.add(palavra)

# Cria um dicionário para contar a frequência de cada palavra em cada documento
freq_palavras = {}
for i, documento in enumerate(documentos):
    palavras = documento.split()
    for palavra in palavras:
        if palavra not in freq_palavras:
            freq_palavras[palavra] = np.zeros(len(documentos), dtype=int)
        freq_palavras[palavra][i] += 1

# Cria a matriz de frequência
matriz = np.zeros((len(documentos), len(todas_palavras)), dtype=int)
for j, palavra in enumerate(todas_palavras):
    if palavra in freq_palavras:
        matriz[:, j] = freq_palavras[palavra]

# Remove as linhas com somente zeros
linhas_nao_nulas = np.any(matriz, axis=1)
matriz = matriz[linhas_nao_nulas]
documentos = [documentos[i] for i in range(len(linhas_nao_nulas)) if linhas_nao_nulas[i]]

# Cria os rótulos de linhas e colunas
rotulos_linhas = np.arange(1, len(documentos)+1, dtype=int).reshape(-1, 1)
rotulos_colunas = np.array(list(todas_palavras))

# Concatena os rótulos de linhas e colunas e a matriz final
matriz_final = np.hstack([np.array([["Documento"]]), rotulos_colunas.reshape(1, -1)])
matriz_final = np.vstack([matriz_final, np.hstack([rotulos_linhas, matriz])])

# Salva a matriz em um arquivo de texto
np.savetxt(nome_arquivo_gerado, matriz_final, delimiter=";", fmt="%s")

print("Arquivo 'BagOfWords.txt' salvo com sucesso.")


In [ ]:
#Gera um arquivo .txt com a matriz TF-IDF
import numpy as np

# Função para calcular o TF-IDF
def calculate_tfidf(freq_palavra_doc, termos_doc, num_documentos, freq_termo_doc):
    if freq_termo_doc > 0 and freq_palavra_doc > 0:
        return (freq_palavra_doc / termos_doc) * np.log(num_documentos / freq_termo_doc)
    else:
        return 0

# Leitura do arquivo de entrada
with open('BagOfWords.txt', 'r') as file:
    lines = file.readlines()

# Extração dos dados do arquivo
num_documentos = len(lines) - 1
num_palavras = len(lines[0].strip().split(';')) - 1
rotulos_palavras = lines[0].strip().split(';')[1:]  # Rótulos das palavras
rotulos_documentos = [line.strip().split(';')[0] for line in lines[1:]]  # Rótulos dos documentos
matriz = np.zeros((num_documentos, num_palavras))

for i in range(1, len(lines)):
    values = lines[i].strip().split(';')[1:]
    for j in range(num_palavras):
        freq_palavra_doc = int(values[j])
        termos_doc = sum(map(int, values))
        freq_termo_doc = sum([1 for line in lines[1:] if int(line.strip().split(';')[j + 1]) > 0])
        matriz[i - 1, j] = calculate_tfidf(freq_palavra_doc, termos_doc, num_documentos, freq_termo_doc)

# Salvando a matriz de resultados em um novo arquivo
with open('TF-IDF.txt', 'w') as file:
    file.write('Documento;' + ';'.join(rotulos_palavras) + '\n')
    for i in range(num_documentos):
        file.write(rotulos_documentos[i] + ';' + ';'.join(map(str, matriz[i, :])))
        file.write('\n')

print("Arquivo 'TF-IDF.txt' salvo com sucesso.")

In [ ]:
#Salva o nome de arquivo gerado como o novo arquivo em uma variável global para futuros usos
nome_arquivo_gerado = "TF-IDF.txt"

In [ ]:
# Esta função faz o filtro pela linha informada pelo usuário na matriz .txt gerada anteriormente 
def ler_matriz(nome_arquivo, num_linha):
    with open(nome_arquivo, 'r') as f:
        linhas = f.readlines()
        valores_linha = linhas[0].strip().split(';')[1:]
        valores_retornados = []
        for i, valor in enumerate(valores_linha):
            coluna = [linha.strip().split(';')[i+1] for linha in linhas[num_linha:num_linha+1]]
            valor_int = float(coluna[0])
            if valor_int != 0:
                valores_retornados += [valor] * int(round(valor_int))
        return valores_retornados


In [ ]:
# Chama a função anterior e imprime uma lista com as repetições das palavras encontradas naquela linha da matriz gerada anteriormente
nome_arquivo = nome_arquivo_gerado
num_linha = int(float(input('Digite o número da linha desejada: ')))
valores = ler_matriz(nome_arquivo, num_linha)
print(valores)

In [ ]:
# Esta função seleciona o maior valor encontrado dentro da matriz salva em arquivo .txt e retorna a palavra seguido da quantia de usos da mesma
def encontrar_maior_valor(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        linhas = f.readlines()
        valores_linha = linhas[0].strip().split(';')[1:]
        maior_valor = float('-inf')
        posicoes_maior_valor = []
        for i, linha in enumerate(linhas[1:]):
            valores = linha.strip().split(';')[1:]
            for j, valor in enumerate(valores):
                valor_int = float(valor)
                if valor_int > maior_valor:
                    maior_valor = valor_int
                    posicoes_maior_valor = [(i+1, j+1)]  # +1 pois estamos ignorando a primeira linha e coluna
                elif valor_int == maior_valor:
                    posicoes_maior_valor.append((i+1, j+1))
        resultados = []
        for posicao in posicoes_maior_valor:
            string_maior_valor = valores_linha[posicao[1]-1]  # -1 pois a posição começa em 1
            resultados.append(f'{string_maior_valor} = {maior_valor}')
        return resultados


In [ ]:
# Imprime o resultado da função acima passando o nome do arquivo gerado como parâmetro
print(encontrar_maior_valor(nome_arquivo_gerado))

['LTD = 8.48446336679332', 'Væring = 8.48446336679332', 'Secondly = 8.48446336679332', '5) = 8.48446336679332', '786-802) = 8.48446336679332', '787 = 8.48446336679332', '66) = 8.48446336679332', '21) = 8.48446336679332', '46-7) = 8.48446336679332', '58) = 8.48446336679332', 'Montgomery) = 8.48446336679332', 'Knútr) = 8.48446336679332', 'York) = 8.48446336679332', "'Brunanburh = 8.48446336679332", 'Dumfriesshire = 8.48446336679332', 'ransomed = 8.48446336679332', 'Ghent = 8.48446336679332', 'Amiens = 8.48446336679332', 'Chartres = 8.48446336679332', 'Tours = 8.48446336679332', 'Blois = 8.48446336679332', 'Orléans = 8.48446336679332', 'Poitiers = 8.48446336679332', 'Limoges = 8.48446336679332', 'Berno = 8.48446336679332', 'Coblentz = 8.48446336679332', 'Aachen = 8.48446336679332', 'Later = 8.48446336679332', 'Eiríkr) = 8.48446336679332', 'Lisieux = 8.48446336679332', 'Seez = 8.48446336679332', '912) = 8.48446336679332', 'Ívarr) = 8.48446336679332', '_knúi = 8.48446336679332', 'knúi_ = 8.

In [ ]:
# Esta função seleciona o menor valor encontrado dentro da matriz salva em arquivo .txt e retorna a palavra seguido da quantia de usos da mesma
def encontrar_menor_valor(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        linhas = f.readlines()
        valores_linha = linhas[0].strip().split(';')[1:]
        menor_valor = float('inf')
        posicoes_menor_valor = []
        for i, linha in enumerate(linhas[1:]):
            valores = linha.strip().split(';')[1:]
            for j, valor in enumerate(valores):
                valor_int = float(valor)
                if valor_int > 0 and valor_int < menor_valor:
                    menor_valor = valor_int
                    posicoes_menor_valor = [(i+1, j+1)]  # +1 pois estamos ignorando a primeira linha e coluna
                elif valor_int > 0 and valor_int == menor_valor:
                    posicoes_menor_valor.append((i+1, j+1))

        resultados = []
        for posicao in posicoes_menor_valor:
            string_menor_valor = valores_linha[posicao[1]-1]  # -1 pois a posição começa em 1
            resultados.append(f'{string_menor_valor} = {menor_valor}')

        if resultados:
            # Verifica se existem outras posições com o mesmo valor mínimo
            for i, linha in enumerate(linhas[1:]):
                valores = linha.strip().split(';')[1:]
                for j, valor in enumerate(valores):
                    if float(valor) == menor_valor and (i+1, j+1) not in posicoes_menor_valor:
                        string_menor_valor = valores_linha[j]  # -1 pois a posição começa em 1
                        resultados.append(f'{string_menor_valor} = {menor_valor}')
            return resultados
        else:
            return 'Não há valores diferentes de zero na matriz.'




In [ ]:
# Imprime o resultado da função acima passando o nome do arquivo gerado como parâmetro
print(encontrar_menor_valor(nome_arquivo_gerado))

['and = 0.028277984212815202']


In [ ]:
import requests
import re
import numpy as np

# Lê o arquivo de texto do URL fornecido
response = requests.get("https://www.gutenberg.org/files/53106/53106-8.txt")
texto = response.text

# Separa o texto em documentos usando "." "," ";" como separadores e substitui "\n" e "\r" por um espaço em branco
separadores = [".", ",", ";"]
texto = texto.replace('\n', ' ').replace('\r', ' ')
documentos = re.split('|'.join(map(re.escape, separadores)), texto)

# Remove os documentos vazios (resultantes de múltiplos caracteres separadores consecutivos)
documentos = list(filter(lambda x: len(x) > 0, documentos))

# Remove os espaços em branco do início e do final dos documentos
documentos = [documento.strip() for documento in documentos]

# Cria um set de todas as palavras encontradas nos documentos
todas_palavras = set()
for documento in documentos:
    palavras = documento.split()
    for palavra in palavras:
        todas_palavras.add(palavra)

# Cria um dicionário para contar a frequência de cada palavra em cada documento
freq_palavras = {}
num_palavras_por_linha = np.zeros(len(documentos), dtype=int)
for i, documento in enumerate(documentos):
    palavras = documento.split()
    num_palavras_por_linha[i] = len(palavras)
    for palavra in palavras:
        if palavra not in freq_palavras:
            freq_palavras[palavra] = np.zeros(len(documentos), dtype=int)
        freq_palavras[palavra][i] += 1

# Cria a matriz de frequência
matriz = np.zeros((len(documentos), len(todas_palavras)), dtype=float)
num_documentos = len(documentos)
for j, palavra in enumerate(todas_palavras):
    if palavra in freq_palavras:
        for i in range(num_documentos):
            freq_palavra_doc = freq_palavras[palavra][i]
            termos_doc = len(documentos[i].split())
            freq_termo_doc = np.count_nonzero(freq_palavras[palavra])
            if freq_termo_doc > 0 and freq_palavra_doc > 0:
                matriz[i, j] = (freq_palavra_doc / termos_doc) * np.log(num_documentos / freq_termo_doc)

# Remove as colunas com somente zeros
colunas_nao_nulas = np.any(matriz, axis=0)
matriz = matriz[:, colunas_nao_nulas]
todas_palavras = list(todas_palavras)
todas_palavras = [todas_palavras[i] for i in range(len(colunas_nao_nulas)) if colunas_nao_nulas[i]]

# Calcula a soma de cada documento
documentos_soma = np.sum(matriz, axis=1)

# Calcula as similaridades entre os documentos usando a fórmula do cosseno
sim_matrix = np.zeros((num_documentos, num_documentos))
for i in range(num_documentos):
    for j in range(num_documentos):
        if i != j and np.linalg.norm(matriz[i])>0 and np.linalg.norm(matriz[j])>0:#modificado, estou testando
            sim_matrix[i, j] = np.dot(matriz[i], matriz[j]) / (np.linalg.norm(matriz[i]) * np.linalg.norm(matriz[j]))
        else:
            sim_matrix[i, j] =0

# Transpõe a matriz para ter cada coluna como um documento
matriz_transposta = matriz.T

# Cria um array para armazenar as somas de cada documento
somas_documentos = np.zeros((len(documentos),), dtype=float)

# Calcula a soma de cada documento
for i in range(len(documentos)):
    somas_documentos[i] = np.sum(matriz[i])

# Cria a matriz de similaridade de cosseno
matriz_similaridade = np.zeros((len(documentos), len(documentos)), dtype=float)

# Calcula a similaridade de cosseno entre cada par de documentos
for i in range(len(documentos)):
    for j in range(len(documentos)):
        if i != j:
            cos_sim = np.dot(matriz_transposta[i], matriz_transposta[j]) / (np.linalg.norm(matriz_transposta[i]) * np.linalg.norm(matriz_transposta[j]))
            matriz_similaridade[i][j] = cos_sim

# Define o valor da diagonal principal como 0
np.fill_diagonal(matriz_similaridade, 0)

# Salva a matriz em um arquivo de texto
np.savetxt('matriz.txt', matriz_similaridade, delimiter=";", fmt="%s")
